In [1]:
#IMPORT PACKAGES
from rdflib import Graph
import rdflib
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d

import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import json
from scipy.spatial.transform import Rotation   
import copy
import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
from scipy.linalg import orthogonal_procrustes
import matplotlib.pyplot as plt
import geomapi.tools.progresstools as pt

from sklearn.cluster import DBSCAN
from PIL import Image

import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
import geomapi.tools.progresstools as pt

#import utils
import context 
import utils as utl
import utils.t8_utils as t8

from typing import Dict, Any, Tuple,List


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [70]:
%autoreload 2

## INPUTS

In [4]:
#paths
path=Path(os.getcwd()).parents[0]

print(path)
input_folder_t4=path/'data'/'t4'/'train' 
input_folder_t6=path/'data'/'t6'/'train'
class_file=path/'data'/'_classes.json'
output_folder=path/'data'/'t8'/ 'train'
os.makedirs(output_folder, exist_ok=True)

#parameters
grid_resolution = 0.01

/home/sdegeyter/Code/Scan-to-BIM-CVPR-2024


Import Classes

In [5]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': 0, 'color': '#9da2ab'}, {'name': 'Floors', 'id': 0, 'temp_id': 1, 'color': '#03c2fc'}, {'name': 'Ceilings', 'id': 1, 'temp_id': 2, 'color': '#e81416'}, {'name': 'Walls', 'id': 2, 'temp_id': 3, 'color': '#ffa500'}, {'name': 'Columns', 'id': 3, 'temp_id': 4, 'color': '#faeb36'}, {'name': 'Doors', 'id': 4, 'temp_id': 5, 'color': '#79c314'}, {'name': 'Windows', 'id': 5, 'temp_id': 6, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


Import Graph

In [6]:
graphfiles=utl.get_list_of_files(input_folder_t6,'.ttl')
for f in graphfiles[:1]: #only read the first one
    print(f'processing {ut.get_filename(f)}...')      
    wallNodes=tl.graph_path_to_nodes(f)
    
    for n in wallNodes:
        n.resource=o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(n.orientedBounds)))

    print(f'{len(wallNodes)} wallNodes detected!')

processing 05_MedOffice_01_F2_small1_walls...
145 wallNodes detected!


Import PCD

In [7]:
pcdfiles=utl.get_list_of_files(input_folder_t4,'.laz')

for f in pcdfiles[:1]: #only read the first one
    pcdNodes=[]
    
    # check if las/pcd variable is already defined    
    print(f'processing {ut.get_filename(f)}...')      
    las = laspy.read(f) if 'las' not in globals() else las
    pcd=gmu.las_to_pcd(las,getNormals=True) if 'pcd' not in globals() else pcd # this is the slowest step
    
    #seperate initial objects
    for c in class_dict['classes']:
        if c['id'] in [255,0,1,2,3,4,5]:
            idx=np.where((las['classes']==c['id']))[0]
            class_pcd=pcd.select_by_index(idx)
            object_labels=las['objects'][idx]
            
            for j in np.unique(object_labels):
                indices=np.where(object_labels==j)[0]
                object_pcd=class_pcd.select_by_index(indices)
                pcdNodes.append(PointCloudNode(resource=object_pcd,
                                            class_id=c['id'],
                                            object_id=j,
                                            color=ut.random_color(),
                                            name=c['name']+f'_{str(j)}'))
                
            
            #all further processing will be placed here (or in functions)!
            print( c['name'], f': {len(pcdNodes)} Nodes found')    

processing 05_MedOffice_01_F2_small1...
Unassigned : 1 Nodes found
Floors : 2 Nodes found
Ceilings : 3 Nodes found
Walls : 148 Nodes found
Columns : 173 Nodes found
Doors : 289 Nodes found
Windows : 289 Nodes found


In [8]:
clutterPCDNodes=[n for n in pcdNodes if 'Unassigned' in n.subject and type(n)==PointCloudNode]
floorPCDNodes=[n for n in pcdNodes if 'Floors' in n.subject and type(n)==PointCloudNode]
ceilingPCDNodes=[n for n in pcdNodes if 'Ceilings' in n.subject and type(n)==PointCloudNode]
wallPCDNodes=[n for n in pcdNodes if 'Walls' in n.subject and type(n)==PointCloudNode]
columnPCDNodes=[n for n in pcdNodes if 'Columns' in n.subject and type(n)==PointCloudNode]
doorPCDNodes=[n for n in pcdNodes if 'Doors' in n.subject and type(n)==PointCloudNode]
windowPCDNodes=[n for n in pcdNodes if 'Windows' in n.subject and type(n)==PointCloudNode]

print(f'{len(clutterPCDNodes)} clutterNodes detected!')
print(f'{len(floorPCDNodes)} floorsNodes detected!')
print(f'{len(ceilingPCDNodes)} ceilingsNodes detected!')
print(f'{len(wallPCDNodes)} wallNodes detected!')
print(f'{len(columnPCDNodes)} columnNodes detected!')
print(f'{len(doorPCDNodes)} doorNodes detected!')
print(f'{len(windowPCDNodes)} windowNodes detected!')

1 clutterNodes detected!
1 floorsNodes detected!
1 ceilingsNodes detected!
145 wallNodes detected!
25 columnNodes detected!
116 doorNodes detected!
0 windowNodes detected!


match point clouds with graph

In [9]:
for n in clutterPCDNodes+floorPCDNodes+ceilingPCDNodes+columnPCDNodes+doorPCDNodes+windowPCDNodes: # this is quite slow because you iterate through 2 scalar fields every time
    idx=np.where((las['classes']==n.class_id) & (las['objects']==n.object_id))
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(las.xyz[idx])
    red = las['red'][idx]
    green = las['green'][idx]
    blue = las['blue'][idx]
    #if color is 32 bit, only keep 8 bit color
    if red.max()>255:
        red = las['red'][idx] >> 8 & 0xFF
        green = las['green'][idx] >> 8 & 0xFF
        blue = las['blue'][idx] >> 8 & 0xFF
    # if colorspace is [0-255] -> remap to [0-1]
    if red.max() >1:
        red=red/255
        green=green/255
        blue=blue/255
    pcd.colors=o3d.utility.Vector3dVector(np.vstack((red,green,blue)).transpose())

    n.resource=pcd
    n.get_oriented_bounding_box()
    n.orientedBoundingBox.color=[1,0,0]

In [10]:
for n in wallPCDNodes:#+ceilingsNodes+floorsNodes: # this is quite slow because you iterate through 2 scalar fields every time
    idx=np.where((las['classes']==n.class_id) & (las['objects']==n.object_id))
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(las.xyz[idx])
    pcd.paint_uniform_color([0.5,0.5,0.5])
    
    red = las['red'][idx]
    green = las['green'][idx]
    blue = las['blue'][idx]
    #if color is 32 bit, only keep 8 bit color
    if red.max()>255:
        red = las['red'][idx] >> 8 & 0xFF
        green = las['green'][idx] >> 8 & 0xFF
        blue = las['blue'][idx] >> 8 & 0xFF
    # if colorspace is [0-255] -> remap to [0-1]
    if red.max() >1:
        red=red/255
        green=green/255
        blue=blue/255
    
    pcd.colors=o3d.utility.Vector3dVector(np.column_stack((red, green, blue)))

    n.resource=pcd
    n.get_oriented_bounding_box()
    n.orientedBoundingBox.color=[1,0,0] 
    

match PointCloudNodes to BIMNodes

In [53]:
for n in wallNodes:
    n.derivedFrom = next(p for p in wallPCDNodes if p.subject.toPython() in [w.derivedFrom for w in wallNodes])


StopIteration: 

## PROCESSING

Convert dtrings from the graph to arrays

In [54]:
for n in wallNodes:
    n.startpoint = np.asarray(n.startpoint[1:-1].split(), dtype=float)
    n.endpoint = np.asarray(n.endpoint[1:-1].split(), dtype=float)
    n.normal = np.asarray(n.normal[1:-1].split(), dtype=float)
    n.height = float(n.height)
    n.name = n.subject.split('///')[-1]
    n.object_id = n.derivedFrom.object_id
    if n.wallThickness == 0.127:
        n.singleFaced = True
    else:
        n.singleFaced = False
    

AttributeError: 'numpy.ndarray' object has no attribute 'split'

Load the full resolution point cloud for a more accurate result (unlabeled)

In [14]:
# laz = laspy.read(os.path.join(Path(os.getcwd()).parents[0]/'data',"full_resolution_populierenhof.las"))
full_res_point_cloud_o3d = gmu.las_to_pcd(las)

Convert the full point cloud into a mesh and add it to a raycasting scene

In [15]:

#Cut a part out of the full resolution pointcloud
# joined_pcd = full_res_point_cloud_o3d.crop(expanded_bounding_box)
#Create a messh from this point cloud 
octree=pt.pcd_to_octree(full_res_point_cloud_o3d,12) #if octree is None else octree
full_res_point_cloud_o3d = None
mesh=gmu.octree_to_voxelmesh(octree) #if mesh is None else mesh


#Create a identity array containing the color so this can be retrieved afterwards
original_colors=np.asarray(mesh.vertex_colors)
indices=np.asarray(mesh.triangles)[:,0]
triangle_colors=original_colors[indices]
#append black color at the end of the array for the invalid hits
triangle_colors=np.vstack((triangle_colors,np.array([0,0,0])))

# Create raycasting scene
scene = o3d.t.geometry.RaycastingScene()
mesh=o3d.t.geometry.TriangleMesh.from_legacy(mesh)
scene.add_triangles(mesh) 

# Calculate the size of each octree node based on octree depth and overall size
def calculate_node_size(octree_depth, octree_size):
    num_voxels_per_dim = 2 ** octree_depth
    voxel_size = octree_size / num_voxels_per_dim
    return voxel_size

# Example usage:
octree_depth = octree.max_depth  # Example value for max_depth
octree_size = octree.size  # Example size of the octree in world units
voxel_size = calculate_node_size(octree_depth, octree_size)
print("Voxelsize = ", voxel_size)
octree = None

Voxelsize =  0.02052154296875


## Method 2: Using the pointcloud and wall data to create an ortho foto of the wall and use object detection
(Can also be used to retrieve other elements)

In [18]:
image_resolution = 0.01

for n in wallNodes:
    
    length = np.sqrt(np.sum((n.endpoint - n.startpoint)**2))
    surface = length * n.height
    image_size = (int(length / image_resolution), int(n.height / image_resolution))
    n.orthos = []
    

    if not surface < 3 and n.height > 1.5 and length > 0.8:  
        #Create an ortho of the dominant side of the wall      
        ortho = t8.create_wall_ortho(startpoint = n.startpoint, endpoint= n.endpoint, height= n.height, resolution = image_resolution, direction = n.normal, scene=scene, triangle_colors = triangle_colors)
        ortho = t8.fill_black_pixels(ortho, region = 10)
        n.orthos.append(ortho)
        #Also create an ortho of the other side of the wall
        if not n.singleFaced: #Single faced wall only needs one side
            ortho = t8.create_wall_ortho(startpoint = n.startpoint, endpoint= n.endpoint, height= n.height, resolution = image_resolution, direction = -n.normal, scene=scene, triangle_colors = triangle_colors, dominant = False)
            ortho = t8.fill_black_pixels(ortho, region = 10)
            n.orthos.append(ortho)

In [19]:
import torch
from PIL import Image
from torchvision.ops import box_convert

# Grounding DINO
from groundingdino.models import build_model
from groundingdino.util.slconfig import SLConfig
from groundingdino.util.utils import clean_state_dict
from groundingdino.util.inference import annotate, load_image, predict

import supervision as sv

# segment anything
# from segment_anything import build_sam, SamPredictor 
import numpy as np


# diffusers
import torch

from huggingface_hub import hf_hub_download

In [21]:
# Use this command for evaluate the Grounding DINO model
# Or you can download the model by yourself
ckpt_repo_id = "ShilongLiu/GroundingDINO"
ckpt_filenmae = "groundingdino_swinb_cogcoor.pth"
ckpt_config_filename = "GroundingDINO_SwinB.cfg.py"

In [24]:
from torch.cuda.amp import autocast
with autocast():
    groundingdino_model = t8.load_model_hf(ckpt_repo_id, ckpt_filenmae, ckpt_config_filename, device="cuda:1")

final text_encoder_type: bert-base-uncased
Model loaded from /home/sdegeyter/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


In [25]:
count = 0
pointcloud = []


for n in wallNodes:
    TEXT_PROMPT = "Door"
    BOX_TRESHOLD = 0.30
    TEXT_TRESHOLD = 0.5
    
    n.boxes = []
    n.logits = []
    n.phrases = []
    
    if len(n.orthos) > 0:
        
        for ortho in n.orthos:
            boxes = None
            image = load_image(Image.fromarray((ortho * 255).astype(np.uint8)))

            boxes, logits, phrases = predict(
                model=groundingdino_model, 
                image=image, 
                caption=TEXT_PROMPT, 
                box_threshold=BOX_TRESHOLD, 
                text_threshold=TEXT_TRESHOLD
            )
            
            n.boxes.append(boxes)
            n.logits.append(logits)
            n.phrases.append(phrases)                  
                    

2024-05-27 12:50:01.138818: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 12:50:03.153540: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sdegeyter/.conda/envs/pointcept/lib/python3.10/site-packages/cv2/../../lib64:
2024-05-27 12:50:03.153695: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sdegeyter/.conda/envs/pointcept/lib/pyth

In [60]:
doorNodes = []
pointcloud = []

wall = 0
count = 0
image = 0
id_count = 4000
class_id = 5
for n in wallNodes:#[72:73]: #^22:24 interessante muren
    potential_door_boxes_wall = []
    potential_door_info_wall = []
    
    if len(n.orthos) > 0 and (len(n.boxes[0]) > 0 or len(n.boxes[1]) > 0):
        
        for j, boxes in enumerate(n.boxes):
            potential_door_boxes_face = []
            potential_door_info_face = []
            if len(boxes) > 0: 
                for i, box in enumerate(boxes):
                    probability = float(n.logits[j][i])
                    # print("Probability: ", probability)
                    
                    opening_width = round(int(np.asarray(box)[2]*n.orthos[j].shape[1])* image_resolution, 2)
                    # print("Opening Width:", opening_width)
                    
                    opening_height = round(int(np.asarray(box)[3]*n.orthos[j].shape[0]) * image_resolution, 2)
                    # print("Opening height:", opening_height)
                    
            
                    detection_center_u = int(np.asarray(box)[0]*n.orthos[j].shape[1]) * image_resolution
                    detection_center_v = int(np.asarray(box)[1]*n.orthos[j].shape[0]) * image_resolution
                    
                    reference_level = round((n.orthos[j].shape[0]*image_resolution) - (detection_center_v + opening_height/2), 2)
                    # print("Reference Level:", reference_level)
                    
                    score = t8.is_door(probability, opening_width, opening_height, reference_level)

                    if score >= 0.3:
                        # print(f"Door {count}")
                        box1 = copy.deepcopy(box)
                        box = box.unsqueeze(0)

                        if j == 1:
                            box1[0] = 1-box1[0]

                        # image = extract_box_with_margin(n.orthos[j], box[0])
                        # image = image[...,::-1] # BGR to RGB

                        potential_door_boxes_face.append(np.asarray(box1))
                        potential_door_info_face.append([opening_width, opening_height, image, reference_level, score])
                              
            potential_door_boxes_wall.append(potential_door_boxes_face)
            potential_door_info_wall.append(potential_door_info_face)
        #merge largely overlapping bounding boxes
        for face_boxes in potential_door_boxes_wall:
            face_boxes = t8.find_and_merge_high_iou_boxes(face_boxes)
    
        #For Double faced walls look if the detection is found on both sides.
        if len(potential_door_boxes_wall) == len(n.orthos) and not len(potential_door_boxes_wall[0]) == 0 and not len(potential_door_boxes_wall[1]) == 0:
            matches, unmatched_boxes0, unmatched_boxes1 = t8.find_best_matches(potential_door_boxes_wall[0], potential_door_boxes_wall[1], potential_door_info_wall[0], potential_door_info_wall[1])
            
            for id0 in unmatched_boxes0:
                score = potential_door_info_wall[0][id0][4]
                if score > 0.7:
                    detectionbox = np.array([potential_door_boxes_wall[0][id0]])
                    image_resource = t8.extract_box_with_margin(n.orthos[0], detectionbox[0])
                    image_resource = image_resource[...,::-1] # BGR to RGB
                    if t8.calculate_percentage_black_pixels(image_resource) < 0.3:
                        
                        # annotated_frame = annotate(image_source= n.orthos[0], boxes=torch.from_numpy(detectionbox), logits=torch.from_numpy(np.array([score])), phrases=[n.phrases[0][i]])
                        # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                        # Image.fromarray(annotated_frame).save(os.path.join(output_folder,(str(image) + "-" +  str(calculate_percentage_black_pixels(image_resource)) +'-DOOR.png')))
                        # image +=1
                        
                        opening_width = round(int(np.asarray(detectionbox)[0][2]*n.orthos[j].shape[1])* image_resolution, 2)
                        
                        opening_height = round(int(np.asarray(detectionbox)[0][3]*n.orthos[j].shape[0]) * image_resolution, 2)
                        
                        detection_center_u = int(np.asarray(detectionbox)[0][0]*n.orthos[j].shape[1]) * image_resolution
                        detection_center_v = int(np.asarray(detectionbox)[0][1]*n.orthos[j].shape[0]) * image_resolution
                        reference_level = round((n.orthos[j].shape[0]*image_resolution) - (detection_center_v + opening_height/2), 2)

                        boundaryPoints = t8.line_with_width_coordinates(n.startpoint, n.endpoint, detectionbox[0][0], opening_width, reference_level)
            
                        doornode = BIMNode()
                        doornode.name= "Doors_" + str(count)
                        doornode.axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1])
                        doornode.startpoint= boundaryPoints[0]
                        doornode.endpoint= boundaryPoints[1]
                        doornode.doorWidth = np.round(np.linalg.norm(boundaryPoints[0] - boundaryPoints[1]),2)
                        doornode.height = opening_height
                        doornode.doornessScore = bestscore
                        doornode.singleFaced = True
                        doornode.depth = n.wallThickness
                        doornode.object_id = class_id *4000 + count
                        
                        doornode.host = n
                        doorNodes.append(doornode)
                        count += 1 
                
            for id1 in unmatched_boxes1:
                score = potential_door_info_wall[1][id1][4]
                if score > 0.7:
                    detectionbox = np.array([potential_door_boxes_wall[1][id1]])
                    image_resource = t8.extract_box_with_margin(n.orthos[1], detectionbox[0])
                    image_resource = image_resource[...,::-1] # BGR to RGB
                    if t8.calculate_percentage_black_pixels(image_resource) < 0.3:
                        
                        # annotated_frame = annotate(image_source= n.orthos[1], boxes=torch.from_numpy(detectionbox), logits=torch.from_numpy(np.array([score])), phrases=[n.phrases[1][i]])
                        # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                        # Image.fromarray(annotated_frame).save(os.path.join(output_folder,(str(image) + "-" +  str(calculate_percentage_black_pixels(image_resource)) +'-DOOR.png')))
                        # image +=1
                        box = np.asarray(potential_door_boxes_wall[1][id1])
                        box[0] = 1-box[0]
                        potential_door_boxes_wall[1][id1] = box
                        opening_width = round(int(np.asarray(detectionbox)[0][2]*n.orthos[j].shape[1])* image_resolution, 2)
                        
                        opening_height = round(int(np.asarray(detectionbox)[0][3]*n.orthos[j].shape[0]) * image_resolution, 2)
                        
                        detection_center_u = int(np.asarray(detectionbox)[0][0]*n.orthos[j].shape[1]) * image_resolution
                        detection_center_v = int(np.asarray(detectionbox)[0][1]*n.orthos[j].shape[0]) * image_resolution
                        reference_level = round((n.orthos[j].shape[0]*image_resolution) - (detection_center_v + opening_height/2), 2)
                        
                        boundaryPoints = t8.line_with_width_coordinates(n.startpoint, n.endpoint, detectionbox[0][0], opening_width, reference_level)
                        
                        doornode = BIMNode()
                        doornode.name= "Doors_" + str(count)
                        doornode.axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1])
                        doornode.startpoint= boundaryPoints[0]
                        doornode.endpoint= boundaryPoints[1]
                        doornode.doorWidth = np.round(np.linalg.norm(boundaryPoints[0] - boundaryPoints[1]),2)
                        doornode.height = opening_height
                        doornode.doornessScore = bestscore
                        doornode.singleFaced = True
                        doornode.depth = n.wallThickness
                        doornode.object_id = class_id *4000 + count
                        
                        doornode.host = n
                        doorNodes.append(doornode)
                        count += 1 

            for id0, id1, bestscore in matches:
                if not id0 == None and not id1 == None:
                    # compare the parameters of the different matches
                    info0 = potential_door_info_wall[0][id0]
                    info1 = potential_door_info_wall[1][id1]
                    image = info0[2]
                    # # boxestemp = np.array([potential_door_boxes_wall[0][id0],potential_door_boxes_wall[1][id1]])
                    # boxes0 = potential_door_boxes_wall[0][id0]
                    # box = np.asarray(potential_door_boxes_wall[1][id1])
                    # box[0] = 1-box[0]
                    # potential_door_boxes_wall[1][id1] = box
                    # potential_door_boxes_wall[1][id1][0] = 1-potential_door_boxes_wall[1][id1][0]
                    
                    detectionbox = t8.combine_boxes(potential_door_boxes_wall[0][id0], potential_door_boxes_wall[1][id1])              
                    # Image.fromarray(image).save(os.path.join(output_folder,(str(count) +'-DOOR.png')))
                    # annotated_frame = annotate(image_source= n.orthos[0], boxes=torch.from_numpy(detectionbox), logits=torch.from_numpy(np.array([bestscore])), phrases=[n.phrases[0][id0]])
                    # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                    # Image.fromarray(annotated_frame).save(os.path.join(output_folder,(str(image) +'-DOOR.png')))
                    image +=1
                    
                    
                    opening_width = round(int(np.asarray(detectionbox)[0][2]*n.orthos[j].shape[1])* image_resolution, 2)
                    # print("Opening Width:", opening_width)
                    
                    opening_height = round(int(np.asarray(detectionbox)[0][3]*n.orthos[j].shape[0]) * image_resolution, 2)
                    
                    detection_center_u = int(np.asarray(detectionbox)[0][0]*n.orthos[j].shape[1]) * image_resolution
                    detection_center_v = int(np.asarray(detectionbox)[0][1]*n.orthos[j].shape[0]) * image_resolution
                    reference_level = round((n.orthos[j].shape[0]*image_resolution) - (detection_center_v + opening_height/2), 2)
                    
                    # print("%s : Door Score: %s => Referencelevel: %s; Width: %s; height: %s" %((str(count)), bestscore, reference_level, opening_width, opening_height))
                    
                    boundaryPoints = t8.line_with_width_coordinates(n.startpoint, n.endpoint, detectionbox[0][0], opening_width, reference_level)
                    
                    doornode = BIMNode()
                    doornode.name= "Doors_" + str(count)
                    doornode.axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1])
                    doornode.startpoint= boundaryPoints[0]
                    doornode.endpoint= boundaryPoints[1]
                    doornode.doorWidth = np.round(np.linalg.norm(boundaryPoints[0] - boundaryPoints[1]),2)
                    doornode.height = opening_height
                    doornode.doornessScore = bestscore
                    doornode.singleFaced = False
                    doornode.depth = n.wallThickness
                    doornode.object_id = class_id *4000 + count
                    
                    doornode.host = n
                    doorNodes.append(doornode)
                    count += 1 
        else: #For single faced walls 
            for i, box in enumerate(potential_door_boxes_wall[0]):
                score = potential_door_info_wall[0][i][4]
                if score > 0.7:
                    detectionbox = np.array([box])
                    image_resource = t8.extract_box_with_margin(n.orthos[0], detectionbox[0])
                    image_resource = image_resource[...,::-1] # BGR to RGB
                    if t8.calculate_percentage_black_pixels(image_resource) < 0.3:
                        
                        # annotated_frame = annotate(image_source= n.orthos[0], boxes=torch.from_numpy(detectionbox), logits=torch.from_numpy(np.array([score])), phrases=[n.phrases[0][i]])
                        # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                        # Image.fromarray(annotated_frame).save(os.path.join(output_folder,(str(image) + "-" +  str(calculate_percentage_black_pixels(image_resource)) +'-DOOR.png')))
                        # image +=1
                        
                        opening_width = round(int(np.asarray(detectionbox)[0][2]*n.orthos[j].shape[1])* image_resolution, 2)
                        # print("Opening Width:", opening_width)
                        
                        opening_height = round(int(np.asarray(detectionbox)[0][3]*n.orthos[j].shape[0]) * image_resolution, 2)
                        
                        detection_center_u = int(np.asarray(detectionbox)[0][0]*n.orthos[j].shape[1]) * image_resolution
                        detection_center_v = int(np.asarray(detectionbox)[0][1]*n.orthos[j].shape[0]) * image_resolution
                        reference_level = round((n.orthos[j].shape[0]*image_resolution) - (detection_center_v + opening_height/2), 2)
                        
                        # print("%s : Door Score: %s => Referencelevel: %s; Width: %s; height: %s" %((str(count)), bestscore, reference_level, opening_width, opening_height))
                        boundaryPoints = t8.line_with_width_coordinates(n.startpoint, n.endpoint, detectionbox[0][0], opening_width, reference_level)
                        
                        doornode = BIMNode()
                        doornode.name= "Doors_" + str(count)
                        doornode.axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1])
                        doornode.startpoint= boundaryPoints[0]
                        doornode.endpoint= boundaryPoints[1]
                        doornode.doorWidth = np.round(np.linalg.norm(boundaryPoints[0] - boundaryPoints[1]),2)
                        doornode.height = opening_height
                        doornode.doornessScore = bestscore
                        doornode.singleFaced = True
                        doornode.depth = n.wallThickness
                        doornode.object_id = class_id *4000 + count
                                            
                        doornode.host = n
                        doorNodes.append(doornode)
                        count += 1
                     
            for i, box in enumerate(potential_door_boxes_wall[1]):
                score = potential_door_info_wall[1][i][4]
                if score >= 0.7:
                    detectionbox = np.array([box])
                    image_resource = t8.extract_box_with_margin(n.orthos[1], detectionbox[0])
                    image_resource = image_resource[...,::-1] # BGR to RGB
                    if t8.calculate_percentage_black_pixels(image_resource) < 0.3:
                        
                        # annotated_frame = annotate(image_source= n.orthos[1], boxes=torch.from_numpy(detectionbox), logits=torch.from_numpy(np.array([score])), phrases=[n.phrases[1][i]])
                        # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                        # Image.fromarray(annotated_frame).save(os.path.join(output_folder,(str(image) + "-" +  str(calculate_percentage_black_pixels(image_resource)) +'-DOOR.png')))
                        # image +=1
                        
                        opening_width = round(int(np.asarray(detectionbox)[0][2]*n.orthos[j].shape[1])* image_resolution, 2)
                        # print("Opening Width:", opening_width)
                        
                        opening_height = round(int(np.asarray(detectionbox)[0][3]*n.orthos[j].shape[0]) * image_resolution, 2)
                        
                        detection_center_u = int(np.asarray(detectionbox)[0][0]*n.orthos[j].shape[1]) * image_resolution
                        detection_center_v = int(np.asarray(detectionbox)[0][1]*n.orthos[j].shape[0]) * image_resolution
                        reference_level = round((n.orthos[j].shape[0]*image_resolution) - (detection_center_v + opening_height/2), 2)
                        
                        # print("%s : Door Score: %s => Referencelevel: %s; Width: %s; height: %s" %((str(count)), bestscore, reference_level, opening_width, opening_height))
                        boundaryPoints = t8.line_with_width_coordinates(n.startpoint, n.endpoint, detectionbox[0][0], opening_width, reference_level)
                        
                        doornode = BIMNode()
                        doornode.name= "Doors_" + str(count)
                        doornode.axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1])
                        doornode.startpoint= boundaryPoints[0]
                        doornode.endpoint= boundaryPoints[1]
                        doornode.doorWidth = np.round(np.linalg.norm(boundaryPoints[0] - boundaryPoints[1]),2)
                        doornode.height = opening_height
                        doornode.doornessScore = bestscore
                        doornode.singleFaced = True
                        doornode.depth = n.wallThickness
                        doornode.object_id = class_id *4000 + count


                        doornode.host = n
                        doorNodes.append(doornode)
                        count += 1 
    wall += 1

In [76]:
def get_angle_with_x_axis(start_point, end_point):
    # Calculate the direction vector of the line
    direction_vector = np.array(end_point) - np.array(start_point)
    
    # Calculate the angle with the x-axis
    # x-axis direction vector
    x_axis_vector = np.array([1, 0, 0])
    
    # Dot product of direction vector and x-axis vector
    dot_product = np.dot(direction_vector, x_axis_vector)
    
    # Magnitude (length) of the direction vector
    magnitude_direction_vector = np.linalg.norm(direction_vector)
    
    # Magnitude of the x-axis vector (which is 1)
    magnitude_x_axis_vector = np.linalg.norm(x_axis_vector)
    
    # Calculate the cosine of the angle
    cos_theta = dot_product / (magnitude_direction_vector * magnitude_x_axis_vector)
    
    # Calculate the angle in radians
    theta_radians = np.arccos(cos_theta)
    
    # Convert the angle to degrees
    theta_degrees = np.degrees(theta_radians)
    
    return theta_degrees

In [77]:
for n in doorNodes:
    pointList=[]
    points=np.asarray(n.axis.points)
    # pointList.extend(points+n.sign*n.normal*n.wallThickness/2)
    pointList.extend(points+n.host.normal*n.host.wallThickness/2)

    # pointList.extend(points-n.sign*n.normal*n.wallThickness/2)
    pointList.extend(points-n.host.normal*n.host.wallThickness/2)

    pointList.extend(np.array(pointList)+np.array([0,0,n.height]))
    pcd=o3d.geometry.PointCloud(points=o3d.utility.Vector3dVector(pointList))

    box=pcd.get_oriented_bounding_box()
    n.resource = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(box)
    n.resource.paint_uniform_color(ut.literal_to_array(n.host.color))
    n.doorBox=o3d.geometry.LineSet.create_from_oriented_bounding_box(box)
    n.doorBox.paint_uniform_color([0,0,1])
    
    n.center = t8.compute_center(n.startpoint, n.endpoint)
    n.cartesianTransform = copy.deepcopy(n.host.cartesianTransform)
    n.cartesianTransform[:3,3] = n.center
    n.normal = n.host.normal
    n.rotation = get_angle_with_x_axis(n.startpoint, n.endpoint)

    print(f'name: {n.name}, score: {n.doornessScore} ,doorWidth: {n.doorWidth}, height: {n.height}')

89.78502280505978
name: Doors_0, score: 0.9695304682379797 ,doorWidth: 0.87, height: 2.12
1.3821368877184934
name: Doors_1, score: 0.9693370941148303 ,doorWidth: 0.94, height: 2.14
89.98907740019823
name: Doors_2, score: 0.9629327635675227 ,doorWidth: 0.87, height: 2.12
179.91193955569642
name: Doors_3, score: 0.9746929097926005 ,doorWidth: 0.89, height: 2.1
89.87172489455205
name: Doors_4, score: 0.9696995368446713 ,doorWidth: 0.94, height: 2.14
89.86066241295532
name: Doors_5, score: 0.9699331354029762 ,doorWidth: 0.9, height: 2.11
0.11879854052313533
name: Doors_6, score: 0.9152734712771989 ,doorWidth: 0.96, height: 2.15
89.85136724662975
name: Doors_7, score: 0.9443849406883149 ,doorWidth: 1.03, height: 2.16
179.92320719261494
name: Doors_8, score: 0.9546706772086225 ,doorWidth: 0.73, height: 2.04
89.74200600684131
name: Doors_9, score: 0.8611966507675779 ,doorWidth: 0.84, height: 2.09
89.9198662404139
name: Doors_10, score: 0.9666590800136377 ,doorWidth: 0.88, height: 2.09
89.9198

In [63]:
lines = []
# # Function to calculate the distance between two points
# def calculate_distance(point1, point2):
#     return np.linalg.norm(np.array(point1) - np.array(point2))


avg_doorWidth = np.mean(np.array([e.doorWidth for e in doorNodes]))
#remove single faced doors that are in a 90 degree angle with double faced doors.
dbscan = DBSCAN(eps=1.5*avg_doorWidth, min_samples=1, metric=t8.calculate_distance)
labels = dbscan.fit_predict([n.center for n in doorNodes])

cluster_ids = [-1] * len(doorNodes)
unique_labels = set(labels)

current_cluster_id = 0
for label in unique_labels:
    cluster_lines = [doorNodes[i] for i in range(len(doorNodes)) if labels[i] == label]
    for line in cluster_lines:
        line_index = None
        for i, l in enumerate(doorNodes):
            if np.array_equal(l, line):
                line_index = i
                break
        cluster_ids[line_index] = current_cluster_id
    current_cluster_id += 1

all_clusters_indices = []

# Get unique cluster labels
unique_labels = np.unique(cluster_ids)

# Iterate through each unique cluster label
for cluster_label in unique_labels:
    if not cluster_label == -1:
        cluster_indices = [i for i, label in enumerate(cluster_ids) if label == cluster_label]
        all_clusters_indices.append(cluster_indices)
    else: 
        for i, label in enumerate(cluster_ids):
            if label == cluster_label:
                all_clusters_indices.append([i])
            
FP = []
for cluster_indices in all_clusters_indices:
    if len(cluster_indices) > 1:
        doors_to_check = []
        for i in cluster_indices:
            if doorNodes[i].singleFaced:
                print(f"door: {doorNodes[i].name} will be removed")
                FP.append(doorNodes[i].subject)

doorNodes = [doornode for doornode in doorNodes if doornode.subject not in FP]


door: Doors_13 will be removed
door: Doors_14 will be removed
door: Doors_42 will be removed
door: Doors_48 will be removed


## Ground truth

In [ ]:
groundtruthPath = '/home/sdegeyter/Code/Scan-to-BIM-CVPR-2024/data/t0/05_MedOffice_01_F2_doors.json'
groundtruthNodes = []

with open(groundtruthPath) as gt:
    data = json.load(gt)
    
for item in data:
    doorNode = Node(
        id = item['id'],
        width = item['width'],
        height = item['height'],
        depth = item['depth'],
        center = item['loc'], 
        rotation = item['rotation'], 
        host = item['host_id']
    )
    groundtruthNodes.append(doorNode)
print(len(groundtruthNodes))


62


In [ ]:
#match the graphs with the training data

input_folder_gt=path/'data'/'t1'/'train'

gt_files=utl.get_list_of_files(input_folder_gt,'.ttl')
gt_files_obj=utl.get_list_of_files(input_folder_gt,'.obj')
for f_gt,f_gt_obj in zip(gt_files[:1],gt_files_obj[:1]): #only read the first one
    
    #import objects
    mesh_dict=utl.load_obj_and_create_meshes(f_gt_obj)
    
    #import graph
    pcdNodes_gt=tl.graph_path_to_nodes(graphPath=str(f_gt))
    pcdNodes_gt_doors=[n for n in pcdNodes_gt if n.class_id==4]

    #add objects to the nodes
    for n in pcdNodes_gt_doors:
        mesh=next((mesh for name, mesh in mesh_dict.items() if name == n.name),None)
        n.resource=mesh
        n.lineset=o3d.geometry.LineSet.create_from_triangle_mesh(mesh)
        n.lineset.paint_uniform_color(ut.literal_to_array(n.color))
        #compute the normal from start to end point
        n.resource.compute_triangle_normals()
        
        n.normal =np.asarray( n.resource.triangle_normals)[11]
       
        
        # direction = (ut.literal_to_array(n_gt.end_pt) - ut.literal_to_array(n.start_pt))/np.linalg.norm(ut.literal_to_array(n_gt.end_pt) - ut.literal_to_array(n.start_pt))
        # n.normal = np.cross(direction, np.array([0, 0, 1]))
        # if np.linalg.norm(normal) == 0:
        #     normal = np.cross(direction, np.array([0, 1, 0]))
        # n.normal = normal / np.linalg.norm(normal)
        
        
    print(f'{len(pcdNodes_gt_doors)} pcdNodes_gt_doors detected!') # there are only 144 walls in the training data, yet 161 are found here
    print(f'{len(doorNodes)} doorNodes detected!') # there are only 144 walls in the training data, yet 161 are found here
    
    #match the walls with the ground truth
    for n in doorNodes:
        
        #create lineset
        n.lineset=o3d.geometry.LineSet.create_from_triangle_mesh(n.resource)
        # n.lineset.paint_uniform_color(ut.literal_to_array(n.color))
                
        #find the corresponding ground truth wall 
        pose=n.cartesianTransform[:3,3]
        distances=[np.linalg.norm(pose-n.cartesianTransform[:3,3]) for n in pcdNodes_gt_doors]
        idx=np.argmin(distances)
        n_gt=pcdNodes_gt_doors[idx]
        # n_gt.color=n.color
        n.corresponding_gt=n_gt.name
        n.corresponding_normal=n_gt.normal
        
        #create a line from the n.resource.get_center() along the n.normal in red and along the n_gt.normal in green
        line=o3d.geometry.LineSet()
        line.points=o3d.utility.Vector3dVector([n.resource.get_center(),n.resource.get_center()+n.normal])
        line.lines=o3d.utility.Vector2iVector([[0,1]])
        line.colors=o3d.utility.Vector3dVector([[1,0,0]])
        n.lineset+=line
        line=o3d.geometry.LineSet()
        line.points=o3d.utility.Vector3dVector([n.resource.get_center(),n.resource.get_center()+n_gt.normal])
        line.lines=o3d.utility.Vector2iVector([[0,1]])
        line.colors=o3d.utility.Vector3dVector([[0,1,0]])
        n.lineset+=line
        
        
        #make comparison
        # n.center_diff=np.round(np.min(np.array([np.linalg.norm(ut.literal_to_array(n_gt.loc)-n.center)])),2)
        n.height_diff=np.round(np.abs(n_gt.height-n.height),2)
        n.width_diff=np.round(n_gt.width-n.doorWidth,2)
        print(f'door {n.get_name()}, height_diff of {n.height_diff} (gt height is {np.round(n_gt.height,2)}), width_diff of {n.width_diff} (gt width is {np.round(n_gt.width,2)})')
    print(f'len of elements width_def >0.03: {len([e for e in doorNodes if np.abs(e.width_diff)>0.03])}/{len(doorNodes)}')
    print(f'len of elements height_diff>0.1: {len([e for e in doorNodes if np.abs(e.height_diff)>0.1])}/{len(doorNodes)}')
    print(f'mean height: {np.mean(np.array([e.height for e in doorNodes]))}')
    print(f'mean width: {np.mean(np.array([e.doorWidth for e in doorNodes]))}')

62 pcdNodes_gt_doors detected!
58 doorNodes detected!
door Doors_0, height_diff of 0.09 (gt height is 2.21), width_diff of 0.2 (gt width is 1.07)
door Doors_1, height_diff of 0.07 (gt height is 2.21), width_diff of 0.13 (gt width is 1.07)
door Doors_2, height_diff of 0.09 (gt height is 2.21), width_diff of 0.2 (gt width is 1.07)
door Doors_3, height_diff of 0.11 (gt height is 2.21), width_diff of 0.18 (gt width is 1.07)
door Doors_4, height_diff of 0.07 (gt height is 2.21), width_diff of 0.13 (gt width is 1.07)
door Doors_5, height_diff of 0.1 (gt height is 2.21), width_diff of 0.07 (gt width is 0.97)
door Doors_6, height_diff of 0.06 (gt height is 2.21), width_diff of 0.01 (gt width is 0.97)
door Doors_7, height_diff of 0.05 (gt height is 2.21), width_diff of 0.04 (gt width is 1.07)
door Doors_8, height_diff of 0.17 (gt height is 2.21), width_diff of 0.34 (gt width is 1.07)
door Doors_9, height_diff of 0.12 (gt height is 2.21), width_diff of 0.23 (gt width is 1.07)
door Doors_10, heig

## EXPORT

In [72]:

t8.write_obj_with_submeshes(os.path.join(output_folder,f'{ut.get_filename(f)}_doors.obj') , [n.resource for n in doorNodes], [n.name for n in doorNodes])
    

In [78]:
reform_name='_'.join(ut.get_filename(f).split('_')[:4])+'_doors'

json_data=t8.doors_to_json(doorNodes)
with open(os.path.join(output_folder,reform_name+'.json'), 'w') as file:
    json.dump(json_data, file, indent=4)
print("JSON data written to file:", os.path.join(output_folder,reform_name+'.json'))

JSON data written to file: /home/sdegeyter/Code/Scan-to-BIM-CVPR-2024/data/t8/test/05_MedOffice_01_F2_doors.json
